<a href="https://colab.research.google.com/github/almedida/thesis/blob/main/estimate_m0s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages('pacman')
library(pacman)

p_load("tidyverse", "tmvtnorm" )

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘zoo’, ‘sandwich’, ‘mvtnorm’, ‘gmm’



tmvtnorm installed



In [ ]:
#load dataset on data_df
data_df = read.table("pvalues_data.csv", header= TRUE, sep=",")

pval_raw = as.data.frame(as.matrix(data_df))
pvals1 <- (as.matrix(pval_raw[,1]))
pvals2 <- (as.matrix(pval_raw[,2]))


In [ ]:
length(pvals1)
length(pvals2)

#dim(pvals2)
head(pvals1, 5)

In [ ]:
###################################################################################
###################################################################################
##calc.cutoff slightly modifies the estimate.m0 function to return the appropriate
##p-value cutoff where all p-values greater than the cutoff are assumed to come
##from null cases.

calc.cutoff = function(p, B = 20, max=1){

  m <- length(p)
  m0 <- m
  bin <- c(-0.1, (1:B)/B*max)
  bin.counts=rep(0,B)

  for(i in 1:B){
    bin.counts[i]=sum((p>bin[i])&(p<=bin[i+1]))
  }

  tail.means <- rev(cumsum(rev(bin.counts))/(1:B))
  temp <- bin.counts - tail.means
  index <- min((1:B)[temp <= 0])
  cutoff2 <- (index)/B*max
  if(cutoff2 == 1) {cutoff2 <- 1-1/B}

  return(cutoff2)

}


In [ ]:
cutoff_value1 = calc.cutoff(pvals1, B=20, max=1)
cutoff_value2 = calc.cutoff(pvals2, B=20, max=1)

cutoff = cbind(c(cutoff_value1), c(cutoff_value2))

colnames(cutoff) = c("cutoff_value1", "cutoff_value2")

cutoff

cutoff_value1,cutoff_value2
0.8,0.9


In [ ]:
#filter dataset using cutoff points
#pvalsB and pvalsM are column names (for pvalues) of the data set for this test(to be changed to column names of the loaded data set)

# selecting pvalues in the upper-right quadrant using Histogram Based Method by Orr et al. (2012)
#defined as lambda1 = cutoffvalue1 and lambda2 = cutoffvalue2 for BS and M respectively.
#lambda1, lambda2 - probability of pvalsB and pvalsM in the upper-right quadrant respectively 


p_vals = pval_raw  %>% filter(pvalue_kidney >=cutoff_value1, pvalue_liver>=cutoff_value2)

dim(p_vals)

head(p_vals, 5)


[1] 53  2

,pvalue_kidney,pvalue_liver
,<dbl>,<dbl>
1,0.8367549,0.9597926
2,0.8624033,0.9850292
3,0.9622471,0.9351648
4,0.9191937,0.9953451
5,0.8049365,0.9646580


In [ ]:
# convert selected pvalues to z values
z_val = as.data.frame(qnorm(as.matrix(p_vals), lower.tail = TRUE))
colnames(z_val) = c("zvals1", "zvals2")

head(z_val)


,zvals1,zvals2
,<dbl>,<dbl>
1,0.9812080,1.748284
2,1.0911807,2.170862
3,1.7773792,1.515403
4,1.3996683,2.600462
5,0.8593872,1.807502
6,1.4086173,2.197443


In [ ]:
zvals1 <- (as.data.frame(as.matrix(z_val[,1])))
zvals2 <- (as.data.frame(as.matrix(z_val[,2])))

head(zvals1, 5)

In [ ]:
# convert lambda(truncation points) to z values
z_val_extremums = as.data.frame(qnorm(as.matrix(cbind(c(cutoff_value1,1),c(cutoff_value2,1))), lower.tail = TRUE))

#selecting the lower bounds (minimum z values) of the bivariate z values
min_z1 <- z_val_extremums[1,1]
min_z2 <- z_val_extremums[1,2]

In [ ]:

head(z_val_extremums, 2)

,V1,V2
,<dbl>,<dbl>
1,0.8416212,1.281552
2,Inf,Inf


In [ ]:
###################################################################################
###################################################################################

estimate.m0s <- function(p1, p2, B=20){
  m <- length(p1)

  ##find lambda cutoffs using histogram-based method
  c1 <- calc.cutoff(p1, B=B, max=1)
  c2 <- calc.cutoff(p2, B=B, max=1)

  ##estimate m0 for experiment 1
  ind1 <- (p1>=c1)
  m0.1 <- sum(ind1)/(1-c1)

  ##estimate m0 for experiment 2  
  ind2 <- (p2>=c2)
  m0.2 <- sum(ind2)/(1-c2)


  ##estimate m00
  ind12 <- ind1 & ind2
  nA <- sum(ind12)
  #pA <- (1-c1)*(1-c2)
  #m00 <- nA/pA
  
  
  # density function for each row of the bivariate z values (x) and 
  # estimated parameters(rho)
  density = function(x, rho)
  {
    sigma = matrix(c(1, rho, rho, 1), 2, 2)
    z = dtmvnorm(x, mean = c(0,0), sigma = sigma, lower = c(min_z1, min_z2))
  }
  
  # log likelihood of the joint densities
  log_likelihood_fn = function(rho){
    
    joint_likelihood = z_val %>% split(.$zvals2) %>% map_dfr(~density(c(.$zvals1,.$zvals2),rho))    
    return(-sum(log(joint_likelihood)))
    
  }
  
  #MLE of the log likelihood function
  optimal_rho = optimize(log_likelihood_fn, lower = -1, upper = 1 )
  optimal_rho = as.data.frame(optimal_rho)
  
  #probability of a random variable greater than cutoff values 
  rho = as.numeric(optimal_rho[1])
  pA = pmvnorm(lower=c(min_z1, min_z2), upper=c(Inf, Inf), mean=c(0,0), sigma = matrix(c(1, rho, rho, 1), 2, 2))
  
  m00 <- nA/pA
  
  
  
  ##estimate m11
  m11 <- sum(m - m0.1 - m0.2 + m00)
  
  
  ret <- list()
  ret$ms <- c(m, m0.1, m0.2, m11, m00)
  names(ret$ms) <- c("m", "m0.1", "m0.2", "m11","m00")
  ret$cutoffs <- c(c1, c2)
  return(ret)
}

In [ ]:
estimates <- estimate.m0s(pvals1, pvals2, B=20)
estimates
#write.csv(estimate.m0s, estimate_m0s.csv)

$ms
       m     m0.1     m0.2      m11      m00 
42575.00  8000.00 17120.00 20821.12  3366.12 

$cutoffs
[1] 0.8 0.9

In [ ]:
mos <- as.matrix(estimates)
write.csv(mos, "estimates.csv")

In [ ]:
###########

############


